# Final Project
### Assessing Unemployment Trends and Economic Recovery
### Exploratory Data Analysis: Visualizing and Analyzing Trends
Jeffrey Wong JW4186

In [3]:
import os
import pyspark

conf = pyspark.SparkConf()
conf = conf.setAppName("JW4186_Final")
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040') ## to setup SPARK UI
conf = conf.set('spark.jars', os.environ['GRAPHFRAMES_PATH']) ## graphframes in spark configuration
sc = pyspark.SparkContext(conf=conf)
sc

24/12/09 13:45:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local[*] appName=JW4186_Final>

In [4]:
spark = pyspark.SQLContext(sc)
spark

/opt/conda/envs/bigdata-spark/lib/python3.11/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


24/12/09 13:46:05 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [116]:
covid19_deaths = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("COVID-19_Death_Counts.csv") 

In [117]:
cre_22 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("CRE_22_County.csv") 

In [148]:
education = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("Education.csv") 

In [149]:
unemployment = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("Unemployment.csv") 

Death counts between 1-9 have been suppressed in covid19_deaths in accordance with NCHS confidentiality standards. Can't be deleted since different counties have different suppressed weeks leading to incorrect totals, so approximating with median of 5

In [120]:
import pyspark.sql.functions as f

covid19_deaths = covid19_deaths.withColumn("Week-Ending Date", f.to_date("Week-Ending Date", "MM/dd/yyyy"))
drop_columns = ["Data As Of", "MMWR Week", "Jurisdiction of Occurrence", "Footnote", "Total Deaths"]
covid19_deaths = covid19_deaths.drop(*drop_columns)
covid19_deaths = covid19_deaths.fillna({"COVID-19 Deaths": 5})
covid19_deaths.createOrReplaceTempView("covid_deaths")
covid_deaths = spark.sql("SELECT FIRST(`State`) AS `State`, `FIPS Code` AS `FIPS`, FIRST(`Urban Rural Code`) AS `Classification`, SUM(`COVID-19 Deaths`) AS `Deaths` FROM covid_deaths GROUP BY `FIPS Code`")

In [121]:
cre_22 = cre_22.withColumn("FIPS", cre_22["STATE"] * 1000 + cre_22["COUNTY"])
cre_22.createOrReplaceTempView("cre_22")
covid_deaths.createOrReplaceTempView("covid_deaths")
covid_deaths = spark.sql("SELECT covid_deaths.State, covid_deaths.FIPS, covid_deaths.Classification, covid_deaths.Deaths, cre_22.POPUNI AS Population FROM covid_deaths INNER JOIN cre_22 ON covid_deaths.FIPS = cre_22.FIPS")

In [122]:
covid_deaths.createOrReplaceTempView("covid_deaths")
state_deaths = spark.sql("SELECT State, SUM(Deaths) as Deaths, SUM(Population) as Population FROM covid_deaths GROUP BY State SORT BY State")
state_deaths = state_deaths.withColumn("Percentage", state_deaths["Deaths"] / state_deaths["Population"] * 100)
state_deaths.toPandas()

,State,Deaths,Population,Percentage
0,AK,2595,708750,0.366138
1,AL,30315,4974000,0.609469
2,AR,20108,2989747,0.672565
3,AZ,31798,7258726,0.438066
4,CA,113545,38627136,0.293951
5,CO,19886,5771236,0.344571
6,DC,2248,647023,0.347437
7,DE,3831,1002476,0.382154
8,FL,85964,21964298,0.391381
9,GA,52976,10739778,0.493269


Counting COVID-19 Deaths By State

In [123]:
urban_rural_deaths = spark.sql("SELECT Classification, SUM(Deaths) as Deaths, SUM(Population) as Population FROM covid_deaths GROUP BY Classification SORT BY Population")
urban_rural_deaths = urban_rural_deaths.withColumn("Percentage", urban_rural_deaths["Deaths"] / urban_rural_deaths["Population"] * 100)
urban_rural_deaths.toPandas()

,Classification,Deaths,Population,Percentage
0,Noncore,150333,18193808,0.826287
1,Micropolitan,197043,26612906,0.740404
2,Small metro,183829,29633133,0.620350
3,Medium metro,298750,67225357,0.444401
4,Large fringe metro,289878,84104636,0.344664
5,Large central metro,342764,99147458,0.345711


Counting COVID-19 Deaths by County Rural-Urban Classification

This shows that rural areas were hit harder by COVID-19 than urban areas, despite having less population density

In [150]:
education = education.select("FIPS Code", "State", "Area Name", "Percent of adults with less than a high school diploma, 2018-22", "Percent of adults with a high school diploma only, 2018-22", "Percent of adults with a bachelor's degree or higher, 2018-22")
col_names = ["FIPS"," State"," County", "no_diploma", "high_school", "bachelors_higher"] 
education = education.toDF(*col_names) 

In [151]:
from pyspark.sql.types import DoubleType
import re

education.createOrReplaceTempView("education")
education_covid = spark.sql("SELECT education.*, covid_deaths.Classification, covid_deaths.Deaths, covid_deaths.Population FROM education INNER JOIN covid_deaths ON education.FIPS = covid_deaths.FIPS")
education_covid = education_covid.withColumn("Percentage", education_covid["Deaths"] / education_covid["Population"] * 100)
education_covid.createOrReplaceTempView("education_covid")
income = unemployment.select("FIPS_Code", "State", "Area_Name", "Median_Household_Income_2021")
income = income.dropna()
commaRep = f.udf(lambda x: re.sub(',','', x))
income = income.withColumn("Median_Household_Income_2021", commaRep("Median_Household_Income_2021"))    
income = income.withColumn("Median_Household_Income_2021", income["Median_Household_Income_2021"].cast(DoubleType()))
income.createOrReplaceTempView("income")
demographics_covid = spark.sql("SELECT `education_covid`.*, income.`Median_Household_Income_2021` as median_income FROM education_covid INNER JOIN income ON education_covid.FIPS = income.FIPS_Code")
no_diploma_percentage = demographics_covid.stat.corr("no_diploma", "Percentage")
print(no_diploma_percentage)

0.12593794032269523


In [152]:
high_school_percentage = demographics_covid.stat.corr("high_school", "Percentage")
print(high_school_percentage)

0.22552010319571694


In [153]:
bachelors_higher_percentage = demographics_covid.stat.corr("bachelors_higher", "Percentage")
print(bachelors_higher_percentage)

-0.27237326016326335


In [155]:
demographics_covid.count()
income_percentage = demographics_covid.corr("median_income", "Percentage")
print(income_percentage)

-0.36615085461812136


These 4 values above show that there is weak correlation between demographics' access to different levels of education or median income levels and COVID death rates at a county scale.

In [156]:
gdp = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("lagdp1224.csv") 

In [12]:
from pyspark.sql.types import IntegerType
import re

gdp = gdp.drop("GDP Rank in State")
gdp = gdp.drop("Percent Change Rank in State")
gdp = gdp.dropna()
gdp_cols = ["GDP 2020", "GDP 2021", "GDP 2022", "GDP 2023"]
commaRep = f.udf(lambda x: re.sub(',','', x))
for col in gdp_cols:
    gdp = gdp.withColumn(col, commaRep(col))    
    gdp = gdp.withColumn(col, f.col(col).cast(IntegerType()))
gdp = gdp.withColumn("GDP Average", gdp["GDP 2020"]/4 + gdp["GDP 2021"]/4 + gdp["GDP 2022"]/4 + gdp["GDP 2023"]/4)
gdp = gdp.withColumn("Total Change", gdp["Percent Change 2021"] + gdp["Percent Change 2022"] + gdp["Percent Change 2023"])

In [13]:
gdp.createOrReplaceTempView("gdp")
county_gdp = spark.sql("SELECT gdp.*, cre_22.POPUNI as Population, cre_22.FIPS FROM gdp INNER JOIN cre_22 ON gdp.County = cre_22.NAME")
county_gdp.createOrReplaceTempView("county_gdp")
covid19_deaths.createOrReplaceTempView("covid_deaths")
covid19_deaths = spark.sql("SELECT FIRST(`State`) AS `State`, `FIPS Code` AS `FIPS`, FIRST(`Urban Rural Code`) AS `Classification`, SUM(`COVID-19 Deaths`) AS `Deaths` FROM covid_deaths GROUP BY `FIPS Code`")
covid19_deaths.createOrReplaceTempView("covid_deaths")
county_gdp_deaths = spark.sql("SELECT county_gdp.County, covid_deaths.*, county_gdp.Population, county_gdp.`GDP Average` as gdp_average, county_gdp.`Total Change` as gdp_change FROM covid_deaths INNER JOIN county_gdp ON covid_deaths.FIPS = county_gdp.FIPS")

In [14]:
county_gdp_deaths = county_gdp_deaths.withColumn("gdp_per_person", county_gdp_deaths["Population"] / county_gdp_deaths["gdp_average"])
county_gdp_deaths.createOrReplaceTempView("county_gdp_deaths")
urban_rural_gdp = spark.sql("SELECT Classification, AVG(Population) as avg_population, AVG(Deaths) as avg_deaths, AVG(gdp_average) as gdp_average, AVG(gdp_per_person) as gdp_per_person, AVG(gdp_change) as gdp_change FROM county_gdp_deaths GROUP BY Classification")
urban_rural_gdp = urban_rural_gdp.withColumn("avg_population", f.round(urban_rural_gdp.avg_population, 2))
urban_rural_gdp = urban_rural_gdp.withColumn("avg_deaths", f.round(urban_rural_gdp.avg_deaths, 2))
urban_rural_gdp = urban_rural_gdp.withColumn("gdp_average", f.round(urban_rural_gdp.gdp_average, 0))
urban_rural_gdp.sort("gdp_change").toPandas()

,Classification,avg_population,avg_deaths,gdp_average,gdp_per_person,gdp_change
0,Small metro,83439.02,516.22,4234440.0,0.026126,8.979710
1,Micropolitan,41660.36,307.64,1916497.0,0.024702,9.924763
2,Noncore,13675.61,112.58,585428.0,0.027162,10.107879
3,Medium metro,182616.01,815.80,9777194.0,0.026419,10.615235
4,Large fringe metro,235117.14,813.95,14141474.0,0.026756,11.061782
5,Large central metro,1515344.97,5276.30,133447073.0,0.013447,11.846032


The column gdp_change represents the average percentage change of a county's gdp from 2020 to 2023, grouped by their classification. 
In addition to seeing a trend of large urban areas having lower death rates, these areas are also recovering and improving economically better or quicker than suburban or rural areas.

In [158]:
demographics_covid.createOrReplaceTempView("demographics_covid")
demographics_gdp = spark.sql("SELECT demographics_covid.*, county_gdp_deaths.gdp_average, county_gdp_deaths.gdp_per_person, county_gdp_deaths.gdp_change FROM demographics_covid INNER JOIN county_gdp_deaths ON county_gdp_deaths.FIPS = demographics_covid.FIPS")
demographics_gdp.corr("no_diploma", "gdp_change")

-0.027310405144907762

In [159]:
demographics_gdp.corr("high_school", "gdp_change")

-0.12367006547943489

In [160]:
demographics_gdp.corr("bachelors_higher", "gdp_change")

0.10377321385562134

In [161]:
demographics_gdp.corr("median_income", "gdp_change")

0.0984901715711428

The above 4 values show that there is no correlation between demographics with differing access to various levels of education or median income and the gdp change or economic growth/shrinking at a county scale.

In [15]:
state_gdp = spark.sql("SELECT State, AVG(Population) as avg_population, AVG(Deaths) as avg_deaths, AVG(gdp_average) as gdp_average, AVG(gdp_per_person) as gdp_per_person, AVG(gdp_change) as gdp_change FROM county_gdp_deaths GROUP BY State")
state_gdp = state_gdp.withColumn("avg_population", f.round(state_gdp.avg_population, 2))
state_gdp = state_gdp.withColumn("avg_deaths", f.round(state_gdp.avg_deaths, 2))
state_gdp = state_gdp.withColumn("gdp_average", f.round(state_gdp.gdp_average, 0))
state_gdp.sort("gdp_change").toPandas()

,State,avg_population,avg_deaths,gdp_average,gdp_per_person,gdp_change
0,NV,185614.53,788.06,10806784.0,0.017008,-1.347059
1,ND,14380.49,105.58,1076852.0,0.015523,2.984906
2,WI,80671.36,368.24,4658349.0,0.024231,4.562500
3,NY,312480.37,1422.39,28024323.0,0.021610,5.317742
4,VT,44960.29,183.36,2428648.0,0.024794,5.457143
5,LA,70395.41,398.34,3718797.0,0.027470,5.792188
6,WV,31593.64,263.45,1403601.0,0.032158,6.690909
7,PA,169049.95,854.35,10065218.0,0.022861,6.725758
8,MS,35075.34,313.68,1409298.0,0.034454,6.878049
9,IL,121933.08,496.19,8403404.0,0.025399,7.210784


Same measure of gdp percentage change from 2020-2023 as above, with the counties grouped by state

In [164]:
unemployment = unemployment.dropna()
unemployment_cols = ["Civilian_labor_force_2019", "Civilian_labor_force_2020", "Civilian_labor_force_2022", "Median_Household_Income_2021"]
commaRep = f.udf(lambda x: re.sub(',','', x))
for col in unemployment_cols:
    unemployment = unemployment.withColumn(col, commaRep(col))    
    unemployment = unemployment.withColumn(col, f.col(col).cast(IntegerType()))
unemployment = unemployment.withColumn("Labor_force_change_from_pre-Covid", unemployment["Civilian_labor_force_2022"] - unemployment["Civilian_labor_force_2019"])
unemployment = unemployment.withColumn("Labor_force_change_percentage_from_pre-Covid", (unemployment["Labor_force_change_from_pre-Covid"] / unemployment["Civilian_labor_force_2019"])*100)
unemployment = unemployment.withColumn("Unemployment_rate_change_from_pre-Covid", unemployment["Unemployment_rate_2022"] - unemployment["Unemployment_rate_2019"])
unemployment = unemployment.withColumn("Labor_force_change_from_Covid", unemployment["Civilian_labor_force_2022"] - unemployment["Civilian_labor_force_2020"])
unemployment = unemployment.withColumn("Labor_force_change_percentage_from_Covid", (unemployment["Labor_force_change_from_Covid"] / unemployment["Civilian_labor_force_2020"])*100)
unemployment = unemployment.withColumn("Unemployment_rate_change_from_Covid", unemployment["Unemployment_rate_2022"] - unemployment["Unemployment_rate_2020"])
unemployment = unemployment.select("FIPS_Code", "State", "Area_Name", "Median_Household_Income_2021", "Labor_force_change_from_pre-Covid", "Labor_force_change_percentage_from_pre-Covid", "Unemployment_rate_change_from_pre-Covid", "Labor_force_change_from_Covid", "Labor_force_change_percentage_from_Covid", "Unemployment_rate_change_from_Covid", "Unemployment_rate_2019", "Unemployment_rate_2020", "Unemployment_rate_2022")

In [165]:
unemployment.createOrReplaceTempView("unemployment")
county_unemployment = spark.sql("SELECT county_gdp_deaths.Classification, county_gdp_deaths.Population, county_gdp_deaths.Deaths, unemployment.* FROM unemployment INNER JOIN county_gdp_deaths ON unemployment.FIPS_Code = county_gdp_deaths.FIPS")
county_unemployment.createOrReplaceTempView("county_unemployment")

In [38]:
urban_rural_labor_change_precovid = spark.sql("SELECT Classification, SUM(`Labor_force_change_from_pre-Covid`), AVG(`Labor_force_change_percentage_from_pre-Covid`) as `Labor_force_change_percentage_from_pre-Covid` FROM county_unemployment GROUP BY Classification SORT BY `Labor_force_change_percentage_from_pre-Covid`")
urban_rural_labor_change_precovid.toPandas()

,Classification,sum(Labor_force_change_from_pre-Covid),Labor_force_change_percentage_from_pre-Covid
0,Micropolitan,-174815,-1.455556
1,Noncore,-121603,-1.076279
2,Small metro,-12650,-0.446107
3,Medium metro,365249,0.498251
4,Large central metro,466484,0.932312
5,Large fringe metro,455232,1.245911


As of the most recently available data from 2022, rural and suburban areas have not yet recovered to the labor force volume (amount of people available to work) that was present pre-COVID in 2019, while urban areas have recovered and even grown their labor force.

In [40]:
urban_rural_unemployment_change_precovid = spark.sql("SELECT Classification, AVG(`Unemployment_rate_change_from_pre-Covid`) as `Unemployment_rate_change_from_pre-Covid`, AVG(`Unemployment_rate_2019`), AVG(`Unemployment_rate_2022`) FROM county_unemployment GROUP BY Classification SORT BY `Unemployment_rate_change_from_pre-Covid`")
urban_rural_unemployment_change_precovid.toPandas()

,Classification,Unemployment_rate_change_from_pre-Covid,avg(Unemployment_rate_2019),avg(Unemployment_rate_2022)
0,Noncore,-0.490722,4.085627,3.594905
1,Small metro,-0.354493,3.906377,3.551884
2,Micropolitan,-0.344479,4.077760,3.733281
3,Medium metro,-0.278393,3.898061,3.619668
4,Large fringe metro,-0.162573,3.447368,3.284795
5,Large central metro,0.182258,3.467742,3.650000


With the same data from 2019 to 2022, the unemployment rate change (lower is better) paints a different picture where unemployment rates have dropped much more in rural and suburban areas compared to urban areas. This can be partially accounted for by the higher COVID death rates in rural and suburban areas, but it also suggests that there is a trend of people moving from rural and suburban areas to urban areas since unemployment rates have not dropped much or even risen in the largest urban areas even though the workforce has grown while the opposite is true for the other areas. It can also be seen that unemployment rates were higher in rural areas than in urban areas pre-COVID and they are much closer post-COVID.

In [41]:
state_labor_change_precovid = spark.sql("SELECT State, SUM(`Labor_force_change_from_pre-Covid`), AVG(`Labor_force_change_percentage_from_pre-Covid`) as `Labor_force_change_percentage_from_pre-Covid` FROM county_unemployment GROUP BY State SORT BY `Labor_force_change_percentage_from_pre-Covid`")
state_labor_change_precovid.toPandas()

,State,sum(Labor_force_change_from_pre-Covid),Labor_force_change_percentage_from_pre-Covid
0,MD,-162856,-4.855182
1,IL,-77632,-4.263138
2,NV,3548,-4.182432
3,MS,-27571,-3.535892
4,ME,-22915,-3.529676
5,NM,-10457,-3.326922
6,VT,-11382,-3.266534
7,PA,-93147,-2.957005
8,NY,-236970,-2.821987
9,OH,-136172,-2.704976


Labor force volume and percentage change from pre-COVID in 2019 to the most recent data in 2022, grouped by state.

In [42]:
state_unemployment_change_precovid = spark.sql("SELECT State, AVG(`Unemployment_rate_change_from_pre-Covid`) as `Unemployment_rate_change_from_pre-Covid`, AVG(`Unemployment_rate_2019`), AVG(`Unemployment_rate_2022`) FROM county_unemployment GROUP BY State SORT BY `Unemployment_rate_change_from_pre-Covid`")
state_unemployment_change_precovid.toPandas()

,State,Unemployment_rate_change_from_pre-Covid,avg(Unemployment_rate_2019),avg(Unemployment_rate_2022)
0,AK,-1.913636,6.977273,5.063636
1,MS,-1.882927,6.469512,4.586585
2,LA,-1.547368,5.487719,3.940351
3,AZ,-1.500000,6.566667,5.066667
4,WV,-1.423636,5.803636,4.380000
5,MO,-0.926957,3.582609,2.655652
6,NE,-0.924731,3.112903,2.188172
7,NM,-0.909375,5.334375,4.425000
8,MN,-0.901149,3.973563,3.072414
9,SD,-0.830769,3.176923,2.346154


Unemployment rate change from pre-COVID in 2019 to 2022 grouped by state. Lower is better.

In [167]:
demographics_labor = spark.sql("SELECT demographics_covid.*, county_unemployment.* FROM demographics_covid INNER JOIN county_unemployment ON county_unemployment.FIPS_Code = demographics_covid.FIPS")
demographics_labor.corr("no_diploma", "Labor_force_change_percentage_from_pre-Covid")

-0.0601637849140097

In [168]:
demographics_labor.corr("high_school", "Labor_force_change_percentage_from_pre-Covid")

-0.20329598562104906

In [169]:
demographics_labor.corr("bachelors_higher", "Labor_force_change_percentage_from_pre-Covid")

0.15810961370456836

In [170]:
demographics_labor.corr("median_income", "Labor_force_change_percentage_from_pre-Covid")

0.17617673130456254

In [171]:
demographics_labor.corr("no_diploma", "unemployment_rate_change_from_pre-Covid")

-0.08500753040186333

In [172]:
demographics_labor.corr("high_school", "unemployment_rate_change_from_pre-Covid")

-0.21369047142415556

In [173]:
demographics_labor.corr("bachelors_higher", "unemployment_rate_change_from_pre-Covid")

0.2147410385807859

In [174]:
demographics_labor.corr("median_income", "unemployment_rate_change_from_pre-Covid")

0.25844195497317485

These 8 values above show there's weak to no correlation between demographics with differing levels of education access or median income and changes in the labor force size or unemployment rate at a county scale from pre-COVID in 2019 to post-COVID in 2022.

In [43]:
urban_rural_labor_change_covid = spark.sql("SELECT Classification, SUM(`Labor_force_change_from_Covid`), AVG(`Labor_force_change_percentage_from_Covid`) as `Labor_force_change_percentage_from_Covid` FROM county_unemployment GROUP BY Classification SORT BY `Labor_force_change_percentage_from_Covid`")
urban_rural_labor_change_covid.toPandas()

,Classification,sum(Labor_force_change_from_Covid),Labor_force_change_percentage_from_Covid
0,Micropolitan,23425,0.138754
1,Noncore,17550,0.508034
2,Small metro,178434,1.205953
3,Medium metro,666906,2.075375
4,Large central metro,1259326,2.223254
5,Large fringe metro,1024941,2.697895


This table shows that rural areas also have not experienced as much growth in the labor force since COVID started compared to urban areas.

In [46]:
urban_rural_unemployment_change_covid = spark.sql("SELECT Classification, AVG(`Unemployment_rate_change_from_Covid`) as `Unemployment_rate_change_from_Covid`, AVG(`Unemployment_rate_2019`), AVG(`Unemployment_rate_2020`), AVG(`Unemployment_rate_2022`) FROM county_unemployment GROUP BY Classification SORT BY `unemployment_rate_change_from_Covid`")
urban_rural_unemployment_change_covid.toPandas()

,Classification,Unemployment_rate_change_from_Covid,avg(Unemployment_rate_2019),avg(Unemployment_rate_2020),avg(Unemployment_rate_2022)
0,Large central metro,-5.101613,3.467742,8.751613,3.650000
1,Large fringe metro,-3.704971,3.447368,6.989766,3.284795
2,Medium metro,-3.521884,3.898061,7.141551,3.619668
3,Micropolitan,-3.284385,4.077760,7.017666,3.733281
4,Small metro,-3.273623,3.906377,6.825507,3.551884
5,Noncore,-2.692928,4.085627,6.287833,3.594905


This table shows that unemployment rates have dropped much faster since COVID in urban areas compared to rural areas. From the trend that can be seen in which the unemployment rate rose more in urban areas from 2019 to 2020 but also dropped more from 2020 to 2022, it can be concluded that the workforce and economy overall are more flexible and resilient in urban areas, leading to a faster and more significant recovery.

In [47]:
state_labor_change_covid = spark.sql("SELECT State, SUM(`Labor_force_change_from_Covid`), AVG(`Labor_force_change_percentage_from_Covid`) as `Labor_force_change_percentage_from_Covid` FROM county_unemployment GROUP BY State SORT BY `Labor_force_change_percentage_from_Covid`")
state_labor_change_covid.toPandas()

,State,sum(Labor_force_change_from_Covid),Labor_force_change_percentage_from_Covid
0,MN,-56658,-2.270934
1,PA,-27962,-1.724461
2,MD,-69573,-1.601022
3,KS,1928,-1.404682
4,WI,-12590,-1.298084
5,AL,16363,-1.196974
6,MO,23403,-1.168349
7,MI,-27043,-1.128859
8,NV,43228,-1.070525
9,MS,3994,-1.027212


Labor force change from COVID in 2020 to post-COVID in 2022 grouped by state.

In [48]:
state_unemployment_change_covid = spark.sql("SELECT State, AVG(`Unemployment_rate_change_from_Covid`) as `Unemployment_rate_change_from_Covid`, AVG(`Unemployment_rate_2019`), AVG(`Unemployment_rate_2020`), AVG(`Unemployment_rate_2022`) FROM county_unemployment GROUP BY State SORT BY `unemployment_rate_change_from_Covid`")
state_unemployment_change_covid.toPandas()

,State,Unemployment_rate_change_from_Covid,avg(Unemployment_rate_2019),avg(Unemployment_rate_2020),avg(Unemployment_rate_2022)
0,HI,-11.566667,2.700000,15.133333,3.566667
1,NJ,-5.690476,3.733333,9.595238,3.904762
2,MA,-5.530769,3.315385,9.561538,4.030769
3,RI,-5.360000,3.240000,8.280000,2.920000
4,CA,-5.077193,5.240351,9.954386,4.877193
5,WV,-4.574545,5.803636,8.954545,4.380000
6,MI,-4.419277,5.002410,9.763855,5.344578
7,NY,-4.398387,4.161290,8.030645,3.632258
8,AK,-4.313636,6.977273,9.377273,5.063636
9,LA,-4.203509,5.487719,8.143860,3.940351


Unemployment rate change from pre-COVID in 2019 to COVID in 2020 to post-COVID in 2022 grouped by state.

In [175]:
demographics_labor.corr("no_diploma", "Labor_force_change_percentage_from_Covid")

-0.05515746455502505

In [176]:
demographics_labor.corr("high_school", "Labor_force_change_percentage_from_Covid")

-0.18632885863734472

In [177]:
demographics_labor.corr("bachelors_higher", "Labor_force_change_percentage_from_Covid")

0.13878730076526707

In [178]:
demographics_labor.corr("median_income", "Labor_force_change_percentage_from_Covid")

0.19169078396894892

In [179]:
demographics_labor.corr("no_diploma", "unemployment_rate_change_from_Covid")

-0.07443689222229743

In [181]:
demographics_labor.corr("high_school", "unemployment_rate_change_from_Covid")

0.01609666246085456

In [183]:
demographics_labor.corr("bachelors_higher", "unemployment_rate_change_from_Covid")

-0.07954202600472264

In [184]:
demographics_labor.corr("median_income", "unemployment_rate_change_from_Covid")

-0.027789242564537762

The above 8 values show there's weak to no correlation between demographics with differing levels of education access or median income and changes in the labor force size or unemployment rate at a county scale from COVID in 2020 to post-COVID in 2022.

In [186]:
cre_19 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("CRE_19_County.csv") 

CRE, or community resilience estimate, is a US Census designated index that measures the capacity of individuals and households within a community to prepare, absorb, respond, and recover from a disaster, whether it be an outbreak or pandemic, a weather event, an economic shock, or any other attack or catastrophe. It is meant to be a broad way of measuring overall socio-economic wellness within a community, and it is calculated using a set array of social vulnerabilities. Each county is split into 3 categories: individuals who face 0 social vulnerabilities, individuals who face 1-2 social vulnerabilities, and individuals who face 3+ social vulnerabilities. Two databases are used for this analysis, one from pre-COVID in 2019 and the other from post-COVID in 2022. Comparing these datasets can therefore be used to measure a community's overall socio-economic recovery from COVID to or surpassing pre-COVID levels.

In [200]:
cre_19 = cre_19.withColumn("FIPS", cre_19["STATE"] * 1000 + cre_19["COUNTY"])
cre_19.createOrReplaceTempView("cre_19")
cre = spark.sql("SELECT cre_22.FIPS, cre_22.NAME as county, cre_19.PRED0_PE as rate0_2019, cre_22.PRED0_PE as rate0_2022, cre_19.PRED12_PE as rate12_2019, cre_22.PRED12_PE as rate12_2022, cre_19.PRED3_PE as rate3_2019, cre_22.PRED3_PE as rate3_2022 FROM cre_19 INNER JOIN cre_22 ON cre_19.NAME = cre_22.NAME")
cre.createOrReplaceTempView("cre")
cre_covid = spark.sql("SELECT covid_deaths.*, cre.county, cre.rate0_2019, cre.rate0_2022, cre.rate12_2019, cre.rate12_2022, cre.rate3_2019, cre.rate3_2022 FROM covid_deaths INNER JOIN cre ON covid_deaths.FIPS = cre.FIPS")
cre_covid = cre_covid.withColumn("Percentage", cre_covid["Deaths"] / cre_covid["Population"])
cre_covid = cre_covid.withColumn("recovery0", cre_covid["rate0_2022"] - cre_covid["rate0_2019"])
cre_covid = cre_covid.withColumn("recovery12", cre_covid["rate12_2022"] - cre_covid["rate12_2019"])
cre_covid = cre_covid.withColumn("recovery3", cre_covid["rate3_2022"] - cre_covid["rate3_2019"])
cre_covid.createOrReplaceTempView("cre_covid")

In [193]:
urban_rural_cre_covid = spark.sql("SELECT Classification, AVG(recovery0), AVG(recovery12), AVG(recovery3) FROM cre_covid GROUP BY Classification SORT BY AVG(recovery0)")
urban_rural_cre_covid.toPandas()

,Classification,avg(recovery0),avg(recovery12),avg(recovery3)
0,Large fringe metro,0.354344,0.597268,-0.952131
1,Large central metro,0.460299,0.412687,-0.873582
2,Micropolitan,0.490484,0.691234,-1.181391
3,Medium metro,0.596522,0.713342,-1.309429
4,Small metro,0.864314,0.274762,-1.138515
5,Noncore,0.932222,0.684662,-1.616907


In [195]:
urban_rural_cre_difference = urban_rural_cre_covid.withColumn("difference0-12", urban_rural_cre_covid["avg(recovery0)"] - urban_rural_cre_covid["avg(recovery12)"])
urban_rural_cre_difference = urban_rural_cre_difference.select("Classification", "difference0-12")
urban_rural_cre_difference.sort("difference0-12", ascending=False).toPandas()

,Classification,difference0-12
0,Small metro,0.589552
1,Noncore,0.247560
2,Large central metro,0.047612
3,Medium metro,-0.116821
4,Micropolitan,-0.200750
5,Large fringe metro,-0.242923


The above 2 charts compare the socio-economic recovery of rural areas vs suburbs vs urban areas. The first chart clearly shows that compared to 2019, the percentage of people facing 3+ social vulnerabilities has fallen across the board. This worst-category is also the only category that has fallen, meaning that communities have unilaterally improved by reducing individuals or households facing 3+ vulnerabilities, even compared to pre-COVID levels. The second chart analyzes if more people have gone from 3+ vulnerabilities to none, or to 1-2. A higher number means more people have gone to 0 vulnerabilities. This chart shows that there's not a clear trend going between rural and urban, with numbers swinging between positive and negative as the population classification grows

In [196]:
state_cre_covid = spark.sql("SELECT State, AVG(recovery0), AVG(recovery12), AVG(recovery3) FROM cre_covid GROUP BY State SORT BY AVG(recovery0)")
state_cre_covid.toPandas()

,State,avg(recovery0),avg(recovery12),avg(recovery3)
0,WA,-0.672051,1.475385,-0.803590
1,NY,-0.656774,0.871129,-0.213226
2,ND,-0.467358,2.660755,-2.193208
3,AK,-0.431111,-0.827778,1.258148
4,NJ,-0.421905,0.776190,-0.352381
5,RI,-0.396000,0.844000,-0.450000
6,WY,-0.379565,-0.821739,1.201304
7,PA,-0.368955,0.810746,-0.441791
8,NE,-0.338280,1.111505,-0.772688
9,LA,-0.057656,0.856250,-0.797344


This chart compares the socio-economic recovery of states in relation to pre-COVID levels. It can be seen that most states have a net reduction in individuals or households facing 3+ social vulnerabilities.

In [204]:
demographics_cre = spark.sql("SELECT demographics_covid.*, cre_covid.Population, cre_covid.rate0_2019, cre_covid.rate0_2022, cre_covid.rate12_2019, cre_covid.rate12_2022, cre_covid.rate3_2019, cre_covid.rate3_2022, cre_covid.recovery0, cre_covid.recovery12, cre_covid.recovery3 FROM demographics_covid INNER JOIN cre_covid ON cre_covid.FIPS = demographics_covid.FIPS")
demographics_cre.corr("no_diploma", "rate3_2019")

0.621933528071364

In [207]:
demographics_cre.corr("no_diploma", "rate3_2022")

0.5832848381601891

In [208]:
demographics_cre.corr("no_diploma", "rate0_2019")

-0.5955021214115549

In [210]:
demographics_cre.corr("no_diploma", "rate0_2022")

-0.5777006923287464

In [211]:
demographics_cre.corr("bachelors_higher", "rate3_2019")

-0.5597253211793881

In [212]:
demographics_cre.corr("bachelors_higher", "rate3_2022")

-0.5377012863693027

In [213]:
demographics_cre.corr("bachelors_higher", "rate0_2019")

0.477349133638774

In [214]:
demographics_cre.corr("bachelors_higher", "rate0_2022")

0.47254180914321464

In [215]:
demographics_cre.corr("median_income", "rate3_2019")

-0.7006979079900322

In [216]:
demographics_cre.corr("median_income", "rate3_2022")

-0.6731393562713254

In [217]:
demographics_cre.corr("median_income", "rate0_2019")

0.5988080557361983

In [218]:
demographics_cre.corr("median_income", "rate0_2022")

0.5811665641535272

The above 12 values show that education level/access and median income have a moderately strong correlation to socio-economic vulnerabilities on a county scale. A trend can be seen that counties with more individuals without a high school diploma tend to have more individuals with 3+ vulnerabilities and fewer individuals with 0 vulnerabilities, while counties with more individuals with a bachelor's degree or above tend to have more individuals with 0 vulnerabilities and fewer individuals with 3+ vulnerabilities. In terms of income, the higher the county's median income, the more individuals it tends to have with 0 vulnerabilities and the fewer with 3+ vulnerabilities, and vice versa.

In [219]:
demographics_cre.corr("no_diploma", "recovery0")

0.08066150356219216

In [220]:
demographics_cre.corr("no_diploma", "recovery3")

-0.13812298078087829

In [225]:
demographics_cre.corr("bachelors_higher", "recovery0")

-0.04831098146753027

In [227]:
demographics_cre.corr("bachelors_higher", "recovery3")

0.10492296454047284

In [228]:
demographics_cre.corr("median_income", "recovery0")

-0.08066226539623614

In [230]:
demographics_cre.corr("median_income", "recovery3")

0.13132995872458797

In contrast to the previous 12 values, the 6 values above show no correlation between education levels/access or median income and the extrapolated socio-economic recovery index subtracting pre-COVID 2019 levels from post-COVID 2022 levels (a positive value means a community is doing better socio-economically in 2022 than 2019, and a negative value is the opposite). 